<a href="https://colab.research.google.com/github/SpoilStick/ML-Projects/blob/main/ML_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Implement a Perceptron

In [ ]:
pip install -q keras-tuner scikeras

     |████████████████████████████████| 135 kB 9.7 MB/s 
     |████████████████████████████████| 1.6 MB 39.7 MB/s 


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from seaborn import load_dataset
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from sklearn.model_selection import GridSearchCV

In [ ]:
diabetes = datasets.load_diabetes()

In [ ]:
data = pd.DataFrame(diabetes.data)
features = diabetes.feature_names
data.columns = features

target = diabetes.target

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
def print_stats(model, data, targets):
  predictions = model.predict(data)
  
  r2 = r2_score(targets, predictions)
  RMSE = mean_squared_error(targets, predictions, squared=False)

  print("r2 Coefficient:", r2)
  print("RMSE:", RMSE)

In [ ]:
perceptron = MLPRegressor().fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
print_stats(perceptron, X_train, y_train)

r2 Coefficient: -3.024856917419166
RMSE: 156.38616491094476


# Part 2: Implement a Keras Classifier

In [ ]:
# Data preparation, for simplicity it's all in one cell

# I'm using the same variable names
# I don't really reuse "data" from before
# It is somewhat annoying though
data = load_dataset("titanic")

data.drop("deck", axis=1)

age_dict = {}

for x in data['pclass'].unique():
    column = data.loc[data['pclass']==x]
    median = column['age'].median()
    age_dict[x] = median

def impute_age(pclass):
  if type(pclass) == int:
    return(age_dict[pclass])

  elif type(pclass) == pd.Series:
    for x in pclass:
      x = age_dict[x]
      return pclass

data['age'] = impute_age(data['pclass'])

data.dropna()

data = pd.get_dummies(data=data, columns=["sex", "embarked", "class", "who", "adult_male", "deck", "embark_town", "alive", "alone"])

X = data
y = X.pop('survived')

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Apparently I have to define a function for the KerasClassifier
# I couldn't find much info about it or a different way
# I also couldn't find a way without using meta
def build_model(meta):
  print(meta["target_type_"])

  # create model
  model = Sequential()
  model.add(Input(meta["n_features_in_"]))
  model.add(Dense(meta["n_classes_"]))

  return model

In [ ]:
classifier = KerasClassifier(model=build_model, loss="binary_crossentropy")

In [ ]:
classifier.fit(X_train, y_train)

binary
23/23 [==============================] - 1s 3ms/step - loss: 5.8866


KerasClassifier(
	model=<function build_model at 0x7f8fb75badd0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=binary_crossentropy
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

In [ ]:
y_pred = classifier.predict(X_test)

score = classifier.scorer(y_test, y_pred)
print(score)

6/6 [==============================] - 0s 4ms/step
0.6871508379888268


# Part 3: Implement a Keras Regressor

In [ ]:
# Data preparation
# Rewriting 'data' again
data = pd.read_csv("bike_share_hour.csv")

data = data.drop(columns=["dteday"])
data.dropna()

scalar = StandardScaler()
scalar.fit(data)
scalar.transform(data)

data = data.drop(columns=["instant", "casual", "registered"])

X = data
y = X.pop("cnt")

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=1/3, random_state=42)

In [ ]:
# Using mean_squared_error did not work for the classifier
def build_regressor_model(n_hidden = 2, n_neurons = 100, input_shape = X_train.shape[1:]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    model.compile(loss='mean_squared_error',metrics=['mae'], optimizer='adam')
    return model

In [ ]:
# Keras sequential regressor
# (What's the difference?)
regressor = KerasRegressor(model=build_regressor_model, loss="mean_squared_error")

In [ ]:
# Gives me an error, why?
regressor.fit(X_train, y_train)

363/363 [==============================] - 3s 5ms/step - loss: 31340.8672 - mae: 127.6582


KerasRegressor(
	model=<function build_regressor_model at 0x7f8fb0a773b0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=mean_squared_error
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
)

In [ ]:
y_pred = regressor.predict(X_test)

score = regressor.scorer(y_test, y_pred)
print(score)

182/182 [==============================] - 1s 3ms/step
0.288018610089649


# Part 4: Tune Your Keras Regressor

In [ ]:
print(regressor.get_params().keys())

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs'])


In [ ]:
batch_size = [5, 10, 20]
epochs = [10, 50, 100]

param_grid = dict(batch_size=batch_size, epochs=epochs)

grid_search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train, y_train)

Epoch 1/10
1854/1854 [==============================] - 4s 2ms/step - loss: 24727.1211 - mae: 113.1427
Epoch 2/10
1854/1854 [==============================] - 3s 2ms/step - loss: 16967.7441 - mae: 94.3675
Epoch 3/10
1854/1854 [==============================] - 4s 2ms/step - loss: 15466.6309 - mae: 88.8119
Epoch 4/10
1854/1854 [==============================] - 3s 2ms/step - loss: 14725.9287 - mae: 85.7349
Epoch 5/10
1854/1854 [==============================] - 4s 2ms/step - loss: 13849.7393 - mae: 82.4225
Epoch 6/10
1854/1854 [==============================] - 4s 2ms/step - loss: 13388.3555 - mae: 80.0743
Epoch 7/10
1854/1854 [==============================] - 4s 2ms/step - loss: 12939.3604 - mae: 77.9764
Epoch 8/10
1854/1854 [==============================] - 4s 2ms/step - loss: 12502.6035 - mae: 76.1409
Epoch 9/10
1854/1854 [==============================] - 4s 2ms/step - loss: 11997.9658 - mae: 74.1784
Epoch 10/10
464/464 [==============================] - 1s 1ms/step
Epoch 1/10
185

In [ ]:
grid_search.best_params_

{'batch_size': 10, 'epochs': 50}

In [ ]:
final_model = grid_search.best_estimator_

y_pred = final_model.predict(X_test)
score = final_model.scorer(y_test, y_pred)
print(score)

580/580 [==============================] - 1s 2ms/step
0.8822247657996198
